In [2]:
import pandas as pd
import os

In [3]:
# Read the problems file
problems = pd.read_json('code4bench_no_loops.json')
loop_problems = pd.read_json('code4bench_unfiltered.json')


In [4]:
# For each of the problems make a folder with the index
for i, problem in problems.iterrows():
    # define the range of problems to extract here
    if i > 2000 and i < 2200:
        # Get the context of the problem
        context = problem['context']
        
        # Get the path, start with the current directory and add the folder name
        path = os.path.join(os.getcwd(), "..", "no_loop_problems", f"{i}")

        # Create the folder if it does not exist
        if not os.path.exists(path):
            os.makedirs(path)
        
        # Write the context to a file
        with open(os.path.join(path, 'context.txt'), 'w', encoding="utf-8") as f:
            f.write(context)

In [13]:
index = 68
print(problems.iloc[index])
print(problems.iloc[index]['sourceCode'])
problems.iloc[index]['sourceCode'].replace("\r", " ").replace("\n", " ").replace("\t", " ")

id                                                         2490
sourceCode    #include<stdio.h>\r\nint main()\r\n{\r\nlong l...
fullname                                                    84A
name                                                          A
countline                                                     7
context       A. Toy Army time limit per test 2 seconds memo...
index                                                        68
Name: 2073, dtype: object
#include<stdio.h>
int main()
{
long long int n;
scanf("%I64d",&n);
printf("%I64d",(3*n)/2);
return(0);}


'#include<stdio.h>  int main()  {  long long int n;  scanf("%I64d",&n);  printf("%I64d",(3*n)/2);  return(0);}'

In [12]:
# Add the index to the problems, starting from 0 and going to the length of the problems
problems['index'] = range(len(problems))

problems[50:100]

,id,sourceCode,fullname,name,countline,context,index
1347,1936,#include<stdio.h>\r\n\r\nint main()\r\n{\r\n ...,259B,B,13,B. Little Elephant and Magic Square time limit...,50
1482,2058,#include<stdio.h>\nint main()\n{ char str[100...,281A,A,14,A. Word Capitalization time limit per test 2 s...,51
1580,2108,"#include ""stdio.h""\r\nint a,b,A[1111];\r\nint ...",290B,B,43,B. QR code time limit per test 2 seconds memor...,52
1588,2111,#include<stdio.h>\r\nint main()\r\n{\r\n ch...,290A,A,10,A. Mysterious strings time limit per test 2 se...,53
1648,2163,#include<stdio.h>\r\n#include<math.h>\r\n#incl...,30A,A,37,A. Accounting time limit per test 2 seconds me...,54
1650,2164,#include <stdio.h>\n#include <stdlib.h>\nint y...,30B,B,42,B. Codeforces World Finals time limit per test...,55
1657,2175,#include<stdio.h>\r\n#include<stdlib.h>\r\nint...,32C,C,41,C. Flea time limit per test 2 seconds memory l...,56
1711,2223,#include<stdio.h>\r\nint main() {\r\n int n...,4A,A,9,A. Watermelon time limit per test 1 second mem...,57
1726,2227,#include <stdio.h>\n\nint main()\n{\n i...,40B,B,22,B. Repaintings time limit per test 2 seconds m...,58
1729,2231,#include<stdio.h>\r\n#include<string.h>\r\nmai...,41A,A,17,A. Translation time limit per test 2 seconds m...,59


In [23]:
# Select a random problem from the loop problems
problem = loop_problems.sample()

    
# Does the problem have a loop?
if problem['sourceCode'].str.contains('for|while|do').any():
    for x in problem:
        print(problem[x])
print(problem['sourceCode'].values[0])
        


828    1377
Name: id, dtype: int64
#include<stdio.h>
int main()
{
	int n,a[1000],b[1000],i,j,c,d,e,f,l=0;
	scanf("%d",&n);
	for(i=0;i<n;i++){
	scanf("%d %d",&a[i],&b[i]);
	}
	for(i=0;i<n;i++){
		c=0;
		d=0;
		e=0;
		f=0;
		for(j=0;j<n;j++){
			if(a[j]>a[i]&&b[j]==b[i]){c++;}
	     	if(a[j]<a[i]&&b[j]==b[i]){d++;}
	    		if(a[j]==a[i]&&b[j]<b[i]){e++;}
	    			if(a[j]==a[i]&&b[j]>b[i]){f++;}
	    		}	if(c>0&&d>0&&e>0&&f>0){l++;}
			
		
		}
	printf("%d",l);
	return 0;
	
	}
828    #include<stdio.h>\r\nint main()\r\n{\r\n\tint ...
Name: sourceCode, dtype: object
#include<stdio.h>
int main()
{
	int n,a[1000],b[1000],i,j,c,d,e,f,l=0;
	scanf("%d",&n);
	for(i=0;i<n;i++){
	scanf("%d %d",&a[i],&b[i]);
	}
	for(i=0;i<n;i++){
		c=0;
		d=0;
		e=0;
		f=0;
		for(j=0;j<n;j++){
			if(a[j]>a[i]&&b[j]==b[i]){c++;}
	     	if(a[j]<a[i]&&b[j]==b[i]){d++;}
	    		if(a[j]==a[i]&&b[j]<b[i]){e++;}
	    			if(a[j]==a[i]&&b[j]>b[i]){f++;}
	    		}	if(c>0&&d>0&&e>0&&f>0){l++;}
			
		
		}
	printf("%d",l);
	return 0;